## Covolutional Neural Network
Applying a CNN on MNIST dataset

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tqdm import tqdm

In [7]:
# Data Collection
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST/", one_hot=True)

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [8]:
# Defining the model parameters

# Number of classes to classify the input data
n_classes = 10
# In order to handle the large data size, we will only concentrate on a batch of data
batch_size = 128

# Defining the output as well as the input data
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

In [10]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# Deining the Neural Network Model
def convolutional_neural_network_model(x):
    # Defining the weights for the first hidden layer with the shape no_inputs X n_nodes_h1
    #5x5 convolution, 1 input, 32 features 
    weights = {'W_conv1': tf.Variable(tf.random_normal([5,5,1,32])),
              'W_conv2': tf.Variable(tf.random_normal([5,5,32,64])),
              'W_fc': tf.Variable(tf.random_normal([7*7*64,1024])),
              'out': tf.Variable(tf.random_normal([1024, n_classes]))}
    
    biases = {'B_conv1': tf.Variable(tf.random_normal([32])),
              'B_conv2': tf.Variable(tf.random_normal([64])),
              'B_fc': tf.Variable(tf.random_normal([1024])),
              'out': tf.Variable(tf.random_normal([n_classes]))}
    
    # Reshaping the input images
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    # Creating the first convolutional layer
    conv1 =  tf.nn.relu(conv2d(x,weights['W_conv1']) + biases['B_conv1'])
    conv1 =  maxpool2d(conv1)

    conv2 =  tf.nn.relu(conv2d(conv1,weights['W_conv2']) + biases['B_conv2'])
    conv2 = maxpool2d(conv2)
    
    # Creating the fully connected layers
    fc = tf.reshape(conv2, shape=[-1,7*7*64])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc']) + biases['B_fc'])
    
    output = tf.matmul(fc, weights['out']) + biases['out']

    return output

In [11]:
# Training the neural network
def train_neural_network(x,y):
    prediction = convolutional_neural_network_model(x)
    # Calculating the cost function
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels= y))
    # Optimizing the cost function : Using the AdamOptimizer and the default learning rate of 0.01
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    # Defining the number of epochs : Cycles of feedforward + backprop
    hm_epochs = 10
    
    # Running the tensorflow session
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # Running through the epochs and training the network
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                x_t, y_t = mnist.train.next_batch(batch_size)
                # Run the optimizer on the cost function
                sess.run(optimizer, feed_dict={x: x_t, y: y_t})
            print("Epoch ", epoch, "completed out of ", hm_epochs)
            
        # Testing the network
        correct = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print("Accuracy" , accuracy.eval({x: mnist.test.images, y: mnist.test.labels})) 

In [ ]:
train_neural_network(x,y)